In [ ]:
import torch
import torchvision
print(f"PyTorch version: {torch.__version__}")
print(f"Torchvision version: {torchvision.__version__}")

In [ ]:
!pip install torch==2.2.0 torchvision==0.17.0 python-dotenv faiss-gpu transformers sentence-transformers pandas peft langchain langchain-community langchain-teddynote pymupdf pymupdf4llm accelerate 'autoawq>=0.1.7'  bitsandbytes

In [ ]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

import json
import os
import unicodedata

import torch
import random
from transformers import set_seed

# 시드 설정
seed = 42
random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)
set_seed(seed)

import pandas as pd
from tqdm import tqdm
import pymupdf
import pymupdf4llm
from collections import Counter

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    pipeline,
    BitsAndBytesConfig,
    AwqConfig
)
from accelerate import Accelerator

# Langchain 관련
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter, MarkdownTextSplitter, MarkdownHeaderTextSplitter
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain_teddynote.retrievers import KiwiBM25Retriever
from langchain.retrievers import EnsembleRetriever, MultiQueryRetriever

from langchain.document_loaders import PDFPlumberLoader, PyMuPDFLoader, PyPDFLoader, UnstructuredPDFLoader
from peft import PeftModel

In [ ]:
class Opt:
    def __init__(self):

        # 모델별 설정 딕셔너리
        self.model_configs = {
            "meta-llama/Meta-Llama-3.1-8B-Instruct":
            {
                "quantization_config": None,
                "torch_dtype": "auto",
                "max_token": 256,
            },
            "rtzr/ko-gemma-2-9b-it": {
                "quantization_config": BitsAndBytesConfig(
                    load_in_4bit=True,
                    bnb_4bit_use_double_quant=True,
                    bnb_4bit_quant_type="nf4",
                    bnb_4bit_compute_dtype=torch.bfloat16
                ),
                "torch_dtype": "auto",
                "max_token": 450
            }
        }
        
        # 선택된 모델
        self.llm_model = "meta-llama/Meta-Llama-3.1-8B-Instruct"
        self.llm_model_config = self.model_configs[self.llm_model]
        self.llm_peft = False
        self.llm_peft_checkpoint = "Meta-Llama-3.1-8B-Instruct-daconfinanace"
        
        self.embed_models = ["intfloat/multilingual-e5-base", "jhgan/ko-sbert-nli", "intfloat/multilingual-e5-large"]
        self.embed_model = self.embed_models[2]
        
        self.pdf_loader = "pymupdf"
        
        self.base_directory = "open/"
        self.train_csv_path = os.path.join(self.base_directory, "train.csv")
        self.test_csv_path = os.path.join(self.base_directory, "test.csv")
        self.chunk_size = 512
        self.chunk_overlap = 32
        
        self.ensemble = True
        self.bm25_w = 0.5
        self.faiss_w = 0.5
        
        self.is_submit = True
        
        self.eval_sum_mode = False
        
        self.output_dir = "test_results"
        self.output_csv_file = f"{self.llm_model.split('/')[1]}_{self.embed_model.split('/')[1]}_pdf{self.pdf_loader}_chks{self.chunk_size}_chkovp{self.chunk_overlap}_bm25{self.bm25_w}_faiss{self.faiss_w}_mix_submission.csv"
        
        os.makedirs(self.output_dir, exist_ok=True)
        
    def to_json(self):
        return json.dumps(self.__dict__)
    
        
args=Opt()

In [ ]:
def setup_llm_pipeline():
    tokenizer = AutoTokenizer.from_pretrained(args.llm_model)
    tokenizer.use_default_system_prompt = False
    
    print(tokenizer.eos_token, tokenizer.eos_token_id)
    
    model = AutoModelForCausalLM.from_pretrained(
        args.llm_model,
        quantization_config=args.llm_model_config['quantization_config'],
        torch_dtype=args.llm_model_config['torch_dtype'],
        device_map="auto",
        trust_remote_code=True
    )

    if args.llm_peft:
        adapter_model = PeftModel.from_pretrained(model, args.llm_peft_checkpoint)
    
    text_generation_pipeline = pipeline(
        model=adapter_model if args.llm_peft else model,
        tokenizer=tokenizer,
        task="text-generation",
        return_full_text=False,
        max_new_tokens=args.llm_model_config['max_token'],
    )

    return HuggingFacePipeline(pipeline=text_generation_pipeline)

In [ ]:
llm = setup_llm_pipeline()

In [ ]:
def normalize_path(path):
    return unicodedata.normalize('NFC', path)
    
def format_docs(docs):
    return "\n".join([doc.page_content for doc in docs])

def process_pdf(file_path):
    md_text = pymupdf4llm.to_markdown(file_path)
    
    headers_to_split_on = [
        ("#", "Header 1"),
        ("##", "Header 2"),
        ("###", "Header 3"),
    ]

    md_header_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on, strip_headers=False)
    md_chunks = md_header_splitter.split_text(md_text)

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=args.chunk_size, chunk_overlap=args.chunk_overlap
    )

    splits = text_splitter.split_documents(md_chunks)
    return splits
    
def create_vector_db(chunks, model_path, method='faiss'):
    model_kwargs = {'device': 'cuda'}
    encode_kwargs = {'normalize_embeddings': True}
    embeddings = HuggingFaceEmbeddings(
        model_name=model_path,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs
    )
    db = FAISS.from_documents(chunks, embedding=embeddings)
    return db


In [ ]:
def process_pdfs_from_dataframe(df, base_directory):
    """딕셔너리에 pdf명을 키로해서 DB, retriever 저장"""
    pdf_databases = {}
    unique_paths = df['Source_path'].unique()
    
    for path in tqdm(unique_paths, desc="Processing PDFs"):
        # 경로 정규화 및 절대 경로 생성
        normalized_path = normalize_path(path)
        full_path = os.path.normpath(os.path.join(base_directory, normalized_path.lstrip('./'))) if not os.path.isabs(normalized_path) else normalized_path
        
        pdf_title = os.path.splitext(os.path.basename(full_path))[0]
        print(f"Processing {pdf_title}...")
        
        # PDF 처리 및 벡터 DB 생성
        chunks = process_pdf(full_path)

        db = create_vector_db(chunks, model_path=args.embed_model)
        kiwi_bm25_retriever = KiwiBM25Retriever.from_documents(chunks)
        faiss_retriever = db.as_retriever()
 
        retriever = EnsembleRetriever(
            retrievers=[kiwi_bm25_retriever, faiss_retriever],
            weights=[args.bm25_w, args.faiss_w],
            search_type="mmr",
        )

        # 결과 저장
        pdf_databases[pdf_title] = {
            'db': db,
            'retriever': retriever
        }
        
    return pdf_databases

In [ ]:
base_directory = args.base_directory  # Your Base Directory
train_df = pd.read_csv(args.train_csv_path)
test_df = pd.read_csv(args.test_csv_path)
train_pdf_databases = None
test_pdf_databases = None
if args.is_submit:
    test_pdf_databases = process_pdfs_from_dataframe(test_df, base_directory)
else:
    train_pdf_databases = process_pdfs_from_dataframe(train_df, base_directory)

In [ ]:
df = test_df if args.is_submit else train_df
pdf_databases = test_pdf_databases if args.is_submit else train_pdf_databases

In [ ]:
def normalize_string(s):
    """유니코드 정규화"""
    return unicodedata.normalize('NFC', s)

In [ ]:
# 결과를 저장할 리스트 초기화
results = []

# DataFrame의 각 행에 대해 처리
for _, row in tqdm(df.iterrows(), total=len(df), desc="Answering Questions"):
    # 소스 문자열 정규화
    source = normalize_string(row['Source'])
    question = row['Question']

    # 정규화된 키로 데이터베이스 검색
    normalized_keys = {normalize_string(k): v for k, v in pdf_databases.items()}
    retriever = normalized_keys[source]['retriever']

    template = """
    다음 정보를 바탕으로 질문에 답하세요:
    {context}

    ### 질문:
    {question}
    
    질문의 핵심만 파악하여 간결하게 1-2문장으로 답변하고, 불필요한 설명은 피하며 요구된 정보만 제공하세요.
    
    ### 답변:

    <|eot_id|>
    """
    
    prompt = PromptTemplate.from_template(template) 
   
    # RAG 체인 정의
    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

    # 답변 추론
    print(f"Question: {question}")
    full_response = rag_chain.invoke(question)
    
    print(f"Answer: {full_response}\n")

    # 결과 저장
    results.append({
        "Source": row['Source'],
        "Source_path": row['Source_path'],
        "Question": question,
        "Answer": full_response
    })

In [ ]:
# 제출용 샘플 파일 로드
submit_df = pd.read_csv("./open/sample_submission.csv")

# 생성된 답변을 제출 DataFrame에 추가
submit_df['Answer'] = [item['Answer'] for item in results]
submit_df['Answer'] = submit_df['Answer'].fillna("데이콘")     # 모델에서 빈 값 (NaN) 생성 시 채점에 오류가 날 수 있음 [ 주의 ]

# 결과를 CSV 파일로 저장
submit_df.to_csv(args.output_csv_file, encoding='UTF-8-sig', index=False)